In [21]:
import openai
import os
from openai import OpenAI

# Set your OpenAI API key
openai.api_key = os.environ.get("OPENAI_API_KEY")

client = OpenAI(api_key= os.environ.get("OPENAI_API_KEY"))

# Define the request to GPT-4o-mini
def send_chat_request():
    try:
        # Make the API request using the new interface
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # Replace with the appropriate model name
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": "Hello, world"},
            ],
            temperature=0.7,  # Adjust creativity level
            max_tokens=50  # Adjust as needed
        )
        
        # Print the response
        print("Response from GPT-4o-mini:")
        print(response.choices[0].message.content.strip())
    except Exception as e:
        print(f"An error occurred: {e}")

# Run the function
if __name__ == "__main__":
    send_chat_request()

  


Response from GPT-4o-mini:
Hello! How can I assist you today?


In [6]:
import openai
from openai import OpenAI
import pandas as pd
import os
import re

# Set your OpenAI API key
openai.api_key = os.environ.get("OPENAI_API_KEY")

client = OpenAI(api_key= os.environ.get("OPENAI_API_KEY"))

def process_questions(input_file):
    # Read the input CSV file
    data = pd.read_csv(input_file)
    
    # Ensure there's a "question" column in the CSV
    if "question" not in data.columns:
        raise ValueError("The input file must have a 'question' column.")
    
    # Prepare the output DataFrame
    output_file = pd.DataFrame(columns=["question_id", "answer"])
    output_file['question_id'] = data['question_id']

    # Loop through each question
    for index, row in data.iterrows():
        question = row["question"]
        
        # Make the API request
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # Replace with the correct model name
            messages=[
                {"role": "system", "content": "You are an expert in reliability theory. This is a reliability engineering exam, with multiple choice questions. Answer each question with only one letter. For example, if the anwser is: '[a] 3.5, just answer 'a'"},
                {"role": "user", "content": question},
            ],
            temperature=0.7,  # Adjust as needed
            max_tokens=50  # Adjust as needed
        )
        
        # Extract the letter corresponding to the answer
        #answer = response['choices'][0]['message']['content'].strip()
        answer = response.choices[0].message.content.strip()
        #print(answer)
        predicted_letter = answer[0] # Take the first letter as prediction
        print(predicted_letter)
        
        # Append the result to the DataFrame
        output_file.loc[index, 'answer'] = predicted_letter

    # Save to the output file
    output_file.to_csv("output.csv", index=False)
    print("Predictions saved to output.csv.")

# Example usage
test = "test.csv"  # Path to your input CSV file
train = "train.csv"  # Path to your input CSV file
process_questions(test)

print(pd.read_csv('output.csv'))


b
a
c
a
a
b
d
a
b
a
a
c
b
d
a
c
b
b
a
c
c
a
a
d
Predictions saved to output.csv.
    question_id answer
0             1      b
1             2      a
2             3      c
3             4      a
4             5      a
5             6      b
6             7      d
7             8      a
8             9      b
9            10      a
10           11      a
11           12      c
12           13      b
13           14      d
14           15      a
15           16      c
16           17      b
17           18      b
18           19      a
19           20      c
20           21      c
21           22      a
22           23      a
23           24      d


In [1]:
"""
Average accuracy over five predictions.
"""

import pandas as pd
import pandas.api.types
from sklearn.metrics import accuracy_score
import numpy as np


class ParticipantVisibleError(Exception):
    # If you want an error message to be shown to participants, you must raise the error as a ParticipantVisibleError
    # All other errors will only be shown to the competition host. This helps prevent unintentional leakage of solution data.
    pass



def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    '''
    >>> import pandas as pd
    >>> num_rows = 5
    >>> data = {'question_id': list(range(num_rows))}
    >>> submission = pd.DataFrame(data)
    >>> for i in range(1, 6):
    >>>     column_name = f'prediction_{i}'
    >>>     submission[column_name] = ['a']*num_rows
    >>> solution = pd.DataFrame(data)
    >>> solution['answer'] = ['a']*num_rows
    >>> _ = score(solution.copy(), submission.copy(), 'question_id')
    Mean accuracy: 1.0
    '''

    if row_id_column_name in solution.columns:
        del solution[row_id_column_name]
    else:
        raise ParticipantVisibleError(f"solution.csv must have a column named {row_id_column_name}.")

    if row_id_column_name in submission.columns:
        del submission[row_id_column_name]
    else:
        raise ParticipantVisibleError(f"submission.csv must have a column named {row_id_column_name}.")

    results = []
    if submission.shape[1] != 5:
        raise ParticipantVisibleError(f"submission.csv must have 5 columns containing the 5 predicitons, but has {submission.shape[1]} columns.")


    y_true = solution['answer']
    for i in range(5):
        y_pred = submission.iloc[:, i]
        result = accuracy_score(y_true, y_pred)
        results.append(result)

    mean_acc = np.mean(results)

    print(f'Mean accuracy: {mean_acc}')

    return mean_acc


In [8]:
sub = pd.read_csv('sample_submission.csv')
print(sub)

    question_id answer
0             1      a
1             2      a
2             3      a
3             4      a
4             5      a
5             6      a
6             7      a
7             8      a
8             9      a
9            10      a
10           11      a
11           12      a
12           13      a
13           14      a
14           15      a
15           16      a
16           17      a
17           18      a
18           19      a
19           20      a
20           21      a
21           22      a
22           23      a
23           24      a


In [6]:
df = pd.read_csv('test.csv')
print(df)
print(df.columns)
print(df['question'][3])

    question_id                                           question
0             1  [Question]: 2.  Battery life has been measured...
1             2  [Question]: 3.  Failure rate data is acquired ...
2             3  [Question]: 4.  Each of the following test acc...
3             4  [Question]: 8.  Sample Variance of resistance ...
4             5  [Question]: 10.  50 samples of an electronic c...
5             6  [Question]: 6. A bearing manufacturer has a be...
6             7  [Question]: 7.  For a certain 3-engine airplan...
7             8  [Question]: 3. The time to failure of a compon...
8             9  [Question]: 8.  While the AMSAA-Duane model is...
9            10  [Question]: 10.  A test was done on ultrasonic...
10           11  [Question]: 1. A computer chip manufacturer av...
11           12  [Question]: 2. Given the following data: (wher...
12           13  [Question]: 4. Which of the following can be p...
13           14  [Question]: 5.  A supplier of parts claims it